Clone the respository

In [ ]:
!git clone https://github.com/Zerve-AI/pypelines.git

Installing the pypeline

In [ ]:
import os
folder = ''
os.chdir(f'{folder}/pypelines')

In [ ]:
!pip install .

LIST OF MODELS

MODELS FOR TIMESERIES CLASSIFICATION PROBLEM

In [ ]:
import pypelines.supervised_pipeline as pipe
from pypelines import utils


utils.list_supported_models(model_type='timeseries_classification')

TIMESERIES CLASSIFICATION

Loading the library

In [33]:
from pypelines import utils
import pandas as pd
from sktime.classification.deep_learning.cnn import CNNClassifier
from sktime.datasets import load_unit_test
from pypelines import ts_classification_pipeline as pipe

df_train, df_train_y= load_unit_test(split="train")
df_test, df_test_y = load_unit_test(split="test")
df_train['class']=df_train_y
df_test['class']=df_test_y

SINGLE TIMESERIES CLASSIFICATION

Data Load and Model Selection

In [39]:
tsc = pipe.TSClassificationPipeline(data=df_train,
                                    target_column='class',
                                    models=['CNN'],
                                    test_data=df_test,positive_class='2')

Default Hyperparameters

In [40]:
tsc.get_hyperparameters()
tsc.code_to_clipboard()

Printing Hyperparameters

In [ ]:
print(tsc.model_grid_search_settings(model_name='CNN'))

Updating Hyperparameters

In [ ]:
hyperparameter = {
    'numerical': 
    [{'search': True, 'name': 'n_epochs', 'min': 100, 'max': 2000, 'step': 1000}, 
     {'search': False, 'name': 'batch_size', 'min': 4, 'max': 16, 'step': 4}, 
     {'search': False, 'name': 'kernel_size', 'min': 4, 'max': 16, 'step': 4}, 
     {'search': False, 'name': 'avg_pool_size', 'min': 4, 'max': 16, 'step': 4}, 
     {'search': False, 'name': 'n_conv_layers', 'min': 4, 'max': 16, 'step': 4}, 
     {'search': False, 'name': 'n_conv_layers', 'min': 4, 'max': 16, 'step': 4}], 
     'categorical': []
     }

print(tsc.set_model_grid_search_settings(hyperparam_dict=hyperparameter, model_name='Random Forest Regression'))

Model tranining code generation

Training code for single Timeseries Classification model

In [ ]:

from sktime import *
from sklearn.metrics import accuracy_score


import pandas as pd
import numpy as np


# target dataframe: df_train
target = "class"
features = list(df_train.columns.drop("class"))

# train test split
X_train = df_train[features]
y_train = df_train[target]

X_test = df_test[features]
y_test = df_test[target]

model_comparison_list = []

##### End of Data Processing Pipeline #####



##### Model Pipeline for CNN #####

from sktime.classification.deep_learning.cnn import CNNClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score,make_scorer,f1_score,precision_score,recall_score,roc_auc_score,roc_curve,auc
import matplotlib.pyplot as plt
CNNClassifier_param_grid = {
"n_epochs": np.arange(100, 2000, 1000),
}

CNNClassifier_model = CNNClassifier()

# Create the grid search
CNNClassifier_grid_search = GridSearchCV(estimator=CNNClassifier_model, param_grid=CNNClassifier_param_grid, cv=3, scoring=make_scorer(accuracy_score), verbose=3)
CNNClassifier_grid_search.fit(X_train, y_train)

# Get the best hyperparameters
CNNClassifier_best_estimator = CNNClassifier_grid_search.best_estimator_

# Store results as a dataframe  
CNNClassifier_search_results = pd.DataFrame(CNNClassifier_grid_search.cv_results_)

# Generate Predictions
CNNClassifier_predictions = pd.DataFrame(CNNClassifier_best_estimator.predict(X_test))
CNNClassifier_predictions_prob = CNNClassifier_best_estimator.predict_proba(X_test)
CNNClassifier_predictions_prob_df = pd.DataFrame()
CNNClassifier_predictions_prob_df[CNNClassifier_grid_search.classes_[0]] = CNNClassifier_predictions_prob[:,0]
CNNClassifier_predictions_prob_df[CNNClassifier_grid_search.classes_[1]] = CNNClassifier_predictions_prob[:,1] 


# Generate Model Metrics
CNNClassifier_accuracy = accuracy_score(y_test, CNNClassifier_predictions.iloc[:,0])
CNNClassifier_f1_score = f1_score(y_test, CNNClassifier_predictions.iloc[:,0],pos_label='2')
CNNClassifier_precision = precision_score(y_test, CNNClassifier_predictions.iloc[:,0],pos_label='2')
CNNClassifier_recall = recall_score(y_test, CNNClassifier_predictions.iloc[:,0],pos_label='2')
CNNClassifier_roc_auc_score = roc_auc_score(y_test, CNNClassifier_predictions_prob_df[CNNClassifier_grid_search.classes_[1]])
CNNClassifier_performance_metrics = [['CNNClassifier','accuracy',CNNClassifier_accuracy], 
                                  ['CNNClassifier','f1_score',CNNClassifier_f1_score],
                                  ['CNNClassifier','precision', CNNClassifier_precision],
                                  ['CNNClassifier','recall', CNNClassifier_recall],
                                  ['CNNClassifier','roc_auc_score', CNNClassifier_roc_auc_score]]
CNNClassifier_performance_metrics = pd.DataFrame(CNNClassifier_performance_metrics, columns=['model','metric', 'value'])
fpr, tpr, thresholds = roc_curve(y_test, CNNClassifier_predictions_prob_df[CNNClassifier_grid_search.classes_[1]],pos_label='2')
roc_auc = auc(fpr, tpr)

# ROC Curve plot
CNNClassifier_roc_auc_plot, CNNClassifier_roc_auc_plot_ax = plt.subplots()
CNNClassifier_roc_auc_plot_ax.plot(fpr, tpr, label=f'ROC curve (AUC = {roc_auc:.4f})')
CNNClassifier_roc_auc_plot_ax.plot([0, 1], [0, 1], 'r--', label='Random guess')
# Set axis labels and title
CNNClassifier_roc_auc_plot_ax.set_xlabel('False Positive Rate')
CNNClassifier_roc_auc_plot_ax.set_ylabel('True Positive Rate')
CNNClassifier_roc_auc_plot_ax.set_title(f'CNNClassifier ROC Curve')
# Add legend
CNNClassifier_roc_auc_plot_ax.legend()


print(CNNClassifier_performance_metrics[CNNClassifier_performance_metrics['metric'] == 'roc_auc_score'])
model_comparison_list.append(CNNClassifier_performance_metrics)##### Model Metrics CNN #####
table = pd.concat(model_comparison_list)
table = table.sort_values(by=['value'], ascending=False)
table = table[table['metric'] == 'roc_auc_score']
print(table)
print(f"The best model is {table['model'].iloc[0]} with {table['value'].iloc[0]} as {table['metric'].iloc[0]}")

# Predict test data using the best model
test_predictions = eval(table['model'].iloc[0]+"_best_estimator").predict(X_test)
print('Predictions from best model are stored in test_predictions')
##### End of Model Pipeline for CNN #####

Multiple Timeseries Classification

In [46]:
tsc = pipe.TSClassificationPipeline(data=df_train,
                                    target_column='class',
                                    models=['CNN', 'TAPNET'],
                                    test_data=df_test,positive_class='2')


Default Hyperparameters

In [ ]:
tsc.get_hyperparameters()


Model tranining code generation

In [48]:
tsc.code_to_clipboard()

Training code for multiple Timeseries Classification model

In [ ]:

from sktime import *
from sklearn.metrics import accuracy_score


import pandas as pd
import numpy as np


# target dataframe: df_train
target = "class"
features = list(df_train.columns.drop("class"))

# train test split
X_train = df_train[features]
y_train = df_train[target]

X_test = df_test[features]
y_test = df_test[target]

model_comparison_list = []

##### End of Data Processing Pipeline #####


##### Model Pipeline for CNN #####

from sktime.classification.deep_learning.cnn import CNNClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score,make_scorer,f1_score,precision_score,recall_score,roc_auc_score,roc_curve,auc
import matplotlib.pyplot as plt
CNNClassifier_param_grid = {
"n_epochs": np.arange(100, 2000, 1000),
}

CNNClassifier_model = CNNClassifier()

# Create the grid search
CNNClassifier_grid_search = GridSearchCV(estimator=CNNClassifier_model, param_grid=CNNClassifier_param_grid, cv=3, scoring=make_scorer(accuracy_score), verbose=3)
CNNClassifier_grid_search.fit(X_train, y_train)

# Get the best hyperparameters
CNNClassifier_best_estimator = CNNClassifier_grid_search.best_estimator_

# Store results as a dataframe  
CNNClassifier_search_results = pd.DataFrame(CNNClassifier_grid_search.cv_results_)

# Generate Predictions
CNNClassifier_predictions = pd.DataFrame(CNNClassifier_best_estimator.predict(X_test))
CNNClassifier_predictions_prob = CNNClassifier_best_estimator.predict_proba(X_test)
CNNClassifier_predictions_prob_df = pd.DataFrame()
CNNClassifier_predictions_prob_df[CNNClassifier_grid_search.classes_[0]] = CNNClassifier_predictions_prob[:,0]
CNNClassifier_predictions_prob_df[CNNClassifier_grid_search.classes_[1]] = CNNClassifier_predictions_prob[:,1] 


# Generate Model Metrics
CNNClassifier_accuracy = accuracy_score(y_test, CNNClassifier_predictions.iloc[:,0])
CNNClassifier_f1_score = f1_score(y_test, CNNClassifier_predictions.iloc[:,0],pos_label='2')
CNNClassifier_precision = precision_score(y_test, CNNClassifier_predictions.iloc[:,0],pos_label='2')
CNNClassifier_recall = recall_score(y_test, CNNClassifier_predictions.iloc[:,0],pos_label='2')
CNNClassifier_roc_auc_score = roc_auc_score(y_test, CNNClassifier_predictions_prob_df[CNNClassifier_grid_search.classes_[1]])
CNNClassifier_performance_metrics = [['CNNClassifier','accuracy',CNNClassifier_accuracy], 
                                  ['CNNClassifier','f1_score',CNNClassifier_f1_score],
                                  ['CNNClassifier','precision', CNNClassifier_precision],
                                  ['CNNClassifier','recall', CNNClassifier_recall],
                                  ['CNNClassifier','roc_auc_score', CNNClassifier_roc_auc_score]]
CNNClassifier_performance_metrics = pd.DataFrame(CNNClassifier_performance_metrics, columns=['model','metric', 'value'])
fpr, tpr, thresholds = roc_curve(y_test, CNNClassifier_predictions_prob_df[CNNClassifier_grid_search.classes_[1]],pos_label='2')
roc_auc = auc(fpr, tpr)

# ROC Curve plot
CNNClassifier_roc_auc_plot, CNNClassifier_roc_auc_plot_ax = plt.subplots()
CNNClassifier_roc_auc_plot_ax.plot(fpr, tpr, label=f'ROC curve (AUC = {roc_auc:.4f})')
CNNClassifier_roc_auc_plot_ax.plot([0, 1], [0, 1], 'r--', label='Random guess')
# Set axis labels and title
CNNClassifier_roc_auc_plot_ax.set_xlabel('False Positive Rate')
CNNClassifier_roc_auc_plot_ax.set_ylabel('True Positive Rate')
CNNClassifier_roc_auc_plot_ax.set_title(f'CNNClassifier ROC Curve')
# Add legend
CNNClassifier_roc_auc_plot_ax.legend()


print(CNNClassifier_performance_metrics[CNNClassifier_performance_metrics['metric'] == 'roc_auc_score'])
model_comparison_list.append(CNNClassifier_performance_metrics)##### End of Model Pipeline for CNN #####
##### Model Pipeline for TAPNET #####

from sktime.classification.deep_learning.tapnet import TapNetClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score,make_scorer,f1_score,precision_score,recall_score,roc_auc_score,roc_curve,auc
import matplotlib.pyplot as plt
TapNetClassifier_param_grid = {
"n_epochs": np.arange(100, 2000, 1000),
"use_lstm": ['True'],
"use_cnn": ['True'],
"use_rp": ['True'],
"use_att": ['True'],
}

TapNetClassifier_model = TapNetClassifier()

# Create the grid search
TapNetClassifier_grid_search = GridSearchCV(estimator=TapNetClassifier_model, param_grid=TapNetClassifier_param_grid, cv=3, scoring=make_scorer(accuracy_score), verbose=3)
TapNetClassifier_grid_search.fit(X_train, y_train)

# Get the best hyperparameters
TapNetClassifier_best_estimator = TapNetClassifier_grid_search.best_estimator_

# Store results as a dataframe  
TapNetClassifier_search_results = pd.DataFrame(TapNetClassifier_grid_search.cv_results_)

# Generate Predictions
TapNetClassifier_predictions = pd.DataFrame(TapNetClassifier_best_estimator.predict(X_test))
TapNetClassifier_predictions_prob = TapNetClassifier_best_estimator.predict_proba(X_test)
TapNetClassifier_predictions_prob_df = pd.DataFrame()
TapNetClassifier_predictions_prob_df[TapNetClassifier_grid_search.classes_[0]] = TapNetClassifier_predictions_prob[:,0]
TapNetClassifier_predictions_prob_df[TapNetClassifier_grid_search.classes_[1]] = TapNetClassifier_predictions_prob[:,1] 


# Generate Model Metrics
TapNetClassifier_accuracy = accuracy_score(y_test, TapNetClassifier_predictions.iloc[:,0])
TapNetClassifier_f1_score = f1_score(y_test, TapNetClassifier_predictions.iloc[:,0],pos_label='2')
TapNetClassifier_precision = precision_score(y_test, TapNetClassifier_predictions.iloc[:,0],pos_label='2')
TapNetClassifier_recall = recall_score(y_test, TapNetClassifier_predictions.iloc[:,0],pos_label='2')
TapNetClassifier_roc_auc_score = roc_auc_score(y_test, TapNetClassifier_predictions_prob_df[TapNetClassifier_grid_search.classes_[1]])
TapNetClassifier_performance_metrics = [['TapNetClassifier','accuracy',TapNetClassifier_accuracy], 
                                  ['TapNetClassifier','f1_score',TapNetClassifier_f1_score],
                                  ['TapNetClassifier','precision', TapNetClassifier_precision],
                                  ['TapNetClassifier','recall', TapNetClassifier_recall],
                                  ['TapNetClassifier','roc_auc_score', TapNetClassifier_roc_auc_score]]
TapNetClassifier_performance_metrics = pd.DataFrame(TapNetClassifier_performance_metrics, columns=['model','metric', 'value'])
fpr, tpr, thresholds = roc_curve(y_test, TapNetClassifier_predictions_prob_df[TapNetClassifier_grid_search.classes_[1]],pos_label='2')
roc_auc = auc(fpr, tpr)

# ROC Curve plot
TapNetClassifier_roc_auc_plot, TapNetClassifier_roc_auc_plot_ax = plt.subplots()
TapNetClassifier_roc_auc_plot_ax.plot(fpr, tpr, label=f'ROC curve (AUC = {roc_auc:.4f})')
TapNetClassifier_roc_auc_plot_ax.plot([0, 1], [0, 1], 'r--', label='Random guess')
# Set axis labels and title
TapNetClassifier_roc_auc_plot_ax.set_xlabel('False Positive Rate')
TapNetClassifier_roc_auc_plot_ax.set_ylabel('True Positive Rate')
TapNetClassifier_roc_auc_plot_ax.set_title(f'TapNetClassifier ROC Curve')
# Add legend
TapNetClassifier_roc_auc_plot_ax.legend()


print(TapNetClassifier_performance_metrics[TapNetClassifier_performance_metrics['metric'] == 'roc_auc_score'])
model_comparison_list.append(TapNetClassifier_performance_metrics)##### End of Model Pipeline for TAPNET #####
##### Model Comparison #####
table = pd.concat(model_comparison_list)
table = table.sort_values(by=['value'], ascending=False)
table = table[table['metric'] == 'roc_auc_score']
print(table)
print(f"The best model is {table['model'].iloc[0]} with {table['value'].iloc[0]} as {table['metric'].iloc[0]}")

# Predict test data using the best model
test_predictions = eval(table['model'].iloc[0]+"_best_estimator").predict(X_test)
print('Predictions from best model are stored in test_predictions')

TIMESERIES CLASSIFICATION MODEL - DEFAULT RUN

In [50]:
tsc = pipe.TSClassificationPipeline(data=df_train,
                                    target_column='class',
                                    test_data=df_test,positive_class='2')

Default Hyperparameters

In [ ]:
tsc.get_hyperparameters()
tsc.model_list()

Model tranining code generation for regression default run

In [53]:
tsc.code_to_clipboard()

In [ ]:

from sktime import *
from sklearn.metrics import accuracy_score


import pandas as pd
import numpy as np


# target dataframe: df_train
target = "class"
features = list(df_train.columns.drop("class"))

# train test split
X_train = df_train[features]
y_train = df_train[target]

X_test = df_test[features]
y_test = df_test[target]

model_comparison_list = []

##### End of Data Processing Pipeline #####


##### Model Pipeline for CNN #####

from sktime.classification.deep_learning.cnn import CNNClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score,make_scorer,f1_score,precision_score,recall_score,roc_auc_score,roc_curve,auc
import matplotlib.pyplot as plt
CNNClassifier_param_grid = {
"n_epochs": np.arange(100, 2000, 1000),
}

CNNClassifier_model = CNNClassifier()

# Create the grid search
CNNClassifier_grid_search = GridSearchCV(estimator=CNNClassifier_model, param_grid=CNNClassifier_param_grid, cv=3, scoring=make_scorer(accuracy_score), verbose=3)
CNNClassifier_grid_search.fit(X_train, y_train)

# Get the best hyperparameters
CNNClassifier_best_estimator = CNNClassifier_grid_search.best_estimator_

# Store results as a dataframe  
CNNClassifier_search_results = pd.DataFrame(CNNClassifier_grid_search.cv_results_)

# Generate Predictions
CNNClassifier_predictions = pd.DataFrame(CNNClassifier_best_estimator.predict(X_test))
CNNClassifier_predictions_prob = CNNClassifier_best_estimator.predict_proba(X_test)
CNNClassifier_predictions_prob_df = pd.DataFrame()
CNNClassifier_predictions_prob_df[CNNClassifier_grid_search.classes_[0]] = CNNClassifier_predictions_prob[:,0]
CNNClassifier_predictions_prob_df[CNNClassifier_grid_search.classes_[1]] = CNNClassifier_predictions_prob[:,1] 


# Generate Model Metrics
CNNClassifier_accuracy = accuracy_score(y_test, CNNClassifier_predictions.iloc[:,0])
CNNClassifier_f1_score = f1_score(y_test, CNNClassifier_predictions.iloc[:,0],pos_label='2')
CNNClassifier_precision = precision_score(y_test, CNNClassifier_predictions.iloc[:,0],pos_label='2')
CNNClassifier_recall = recall_score(y_test, CNNClassifier_predictions.iloc[:,0],pos_label='2')
CNNClassifier_roc_auc_score = roc_auc_score(y_test, CNNClassifier_predictions_prob_df[CNNClassifier_grid_search.classes_[1]])
CNNClassifier_performance_metrics = [['CNNClassifier','accuracy',CNNClassifier_accuracy], 
                                  ['CNNClassifier','f1_score',CNNClassifier_f1_score],
                                  ['CNNClassifier','precision', CNNClassifier_precision],
                                  ['CNNClassifier','recall', CNNClassifier_recall],
                                  ['CNNClassifier','roc_auc_score', CNNClassifier_roc_auc_score]]
CNNClassifier_performance_metrics = pd.DataFrame(CNNClassifier_performance_metrics, columns=['model','metric', 'value'])
fpr, tpr, thresholds = roc_curve(y_test, CNNClassifier_predictions_prob_df[CNNClassifier_grid_search.classes_[1]],pos_label='2')
roc_auc = auc(fpr, tpr)

# ROC Curve plot
CNNClassifier_roc_auc_plot, CNNClassifier_roc_auc_plot_ax = plt.subplots()
CNNClassifier_roc_auc_plot_ax.plot(fpr, tpr, label=f'ROC curve (AUC = {roc_auc:.4f})')
CNNClassifier_roc_auc_plot_ax.plot([0, 1], [0, 1], 'r--', label='Random guess')
# Set axis labels and title
CNNClassifier_roc_auc_plot_ax.set_xlabel('False Positive Rate')
CNNClassifier_roc_auc_plot_ax.set_ylabel('True Positive Rate')
CNNClassifier_roc_auc_plot_ax.set_title(f'CNNClassifier ROC Curve')
# Add legend
CNNClassifier_roc_auc_plot_ax.legend()


print(CNNClassifier_performance_metrics[CNNClassifier_performance_metrics['metric'] == 'roc_auc_score'])
model_comparison_list.append(CNNClassifier_performance_metrics)##### End of Model Pipeline for CNN #####
##### Model Pipeline for LSTMFCN #####

from sktime.classification.deep_learning.lstmfcn import LSTMFCNClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score,make_scorer,f1_score,precision_score,recall_score,roc_auc_score,roc_curve,auc
import matplotlib.pyplot as plt
LSTMFCNClassifier_param_grid = {
"n_epochs": np.arange(100, 2000, 1000),
}

LSTMFCNClassifier_model = LSTMFCNClassifier()

# Create the grid search
LSTMFCNClassifier_grid_search = GridSearchCV(estimator=LSTMFCNClassifier_model, param_grid=LSTMFCNClassifier_param_grid, cv=3, scoring=make_scorer(accuracy_score), verbose=3)
LSTMFCNClassifier_grid_search.fit(X_train, y_train)

# Get the best hyperparameters
LSTMFCNClassifier_best_estimator = LSTMFCNClassifier_grid_search.best_estimator_

# Store results as a dataframe  
LSTMFCNClassifier_search_results = pd.DataFrame(LSTMFCNClassifier_grid_search.cv_results_)

# Generate Predictions
LSTMFCNClassifier_predictions = pd.DataFrame(LSTMFCNClassifier_best_estimator.predict(X_test))
LSTMFCNClassifier_predictions_prob = LSTMFCNClassifier_best_estimator.predict_proba(X_test)
LSTMFCNClassifier_predictions_prob_df = pd.DataFrame()
LSTMFCNClassifier_predictions_prob_df[LSTMFCNClassifier_grid_search.classes_[0]] = LSTMFCNClassifier_predictions_prob[:,0]
LSTMFCNClassifier_predictions_prob_df[LSTMFCNClassifier_grid_search.classes_[1]] = LSTMFCNClassifier_predictions_prob[:,1] 


# Generate Model Metrics
LSTMFCNClassifier_accuracy = accuracy_score(y_test, LSTMFCNClassifier_predictions.iloc[:,0])
LSTMFCNClassifier_f1_score = f1_score(y_test, LSTMFCNClassifier_predictions.iloc[:,0],pos_label='2')
LSTMFCNClassifier_precision = precision_score(y_test, LSTMFCNClassifier_predictions.iloc[:,0],pos_label='2')
LSTMFCNClassifier_recall = recall_score(y_test, LSTMFCNClassifier_predictions.iloc[:,0],pos_label='2')
LSTMFCNClassifier_roc_auc_score = roc_auc_score(y_test, LSTMFCNClassifier_predictions_prob_df[LSTMFCNClassifier_grid_search.classes_[1]])
LSTMFCNClassifier_performance_metrics = [['LSTMFCNClassifier','accuracy',LSTMFCNClassifier_accuracy], 
                                  ['LSTMFCNClassifier','f1_score',LSTMFCNClassifier_f1_score],
                                  ['LSTMFCNClassifier','precision', LSTMFCNClassifier_precision],
                                  ['LSTMFCNClassifier','recall', LSTMFCNClassifier_recall],
                                  ['LSTMFCNClassifier','roc_auc_score', LSTMFCNClassifier_roc_auc_score]]
LSTMFCNClassifier_performance_metrics = pd.DataFrame(LSTMFCNClassifier_performance_metrics, columns=['model','metric', 'value'])
fpr, tpr, thresholds = roc_curve(y_test, LSTMFCNClassifier_predictions_prob_df[LSTMFCNClassifier_grid_search.classes_[1]],pos_label='2')
roc_auc = auc(fpr, tpr)

# ROC Curve plot
LSTMFCNClassifier_roc_auc_plot, LSTMFCNClassifier_roc_auc_plot_ax = plt.subplots()
LSTMFCNClassifier_roc_auc_plot_ax.plot(fpr, tpr, label=f'ROC curve (AUC = {roc_auc:.4f})')
LSTMFCNClassifier_roc_auc_plot_ax.plot([0, 1], [0, 1], 'r--', label='Random guess')
# Set axis labels and title
LSTMFCNClassifier_roc_auc_plot_ax.set_xlabel('False Positive Rate')
LSTMFCNClassifier_roc_auc_plot_ax.set_ylabel('True Positive Rate')
LSTMFCNClassifier_roc_auc_plot_ax.set_title(f'LSTMFCNClassifier ROC Curve')
# Add legend
LSTMFCNClassifier_roc_auc_plot_ax.legend()


print(LSTMFCNClassifier_performance_metrics[LSTMFCNClassifier_performance_metrics['metric'] == 'roc_auc_score'])
model_comparison_list.append(LSTMFCNClassifier_performance_metrics)##### End of Model Pipeline for LSTMFCN #####
##### Model Pipeline for FCN #####

from sktime.classification.deep_learning.fcn import FCNClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score,make_scorer,f1_score,precision_score,recall_score,roc_auc_score,roc_curve,auc
import matplotlib.pyplot as plt
FCNClassifier_param_grid = {
"n_epochs": np.arange(100, 2000, 1000),
"activation": ['sigmoid'],
}

FCNClassifier_model = FCNClassifier()

# Create the grid search
FCNClassifier_grid_search = GridSearchCV(estimator=FCNClassifier_model, param_grid=FCNClassifier_param_grid, cv=3, scoring=make_scorer(accuracy_score), verbose=3)
FCNClassifier_grid_search.fit(X_train, y_train)

# Get the best hyperparameters
FCNClassifier_best_estimator = FCNClassifier_grid_search.best_estimator_

# Store results as a dataframe  
FCNClassifier_search_results = pd.DataFrame(FCNClassifier_grid_search.cv_results_)

# Generate Predictions
FCNClassifier_predictions = pd.DataFrame(FCNClassifier_best_estimator.predict(X_test))
FCNClassifier_predictions_prob = FCNClassifier_best_estimator.predict_proba(X_test)
FCNClassifier_predictions_prob_df = pd.DataFrame()
FCNClassifier_predictions_prob_df[FCNClassifier_grid_search.classes_[0]] = FCNClassifier_predictions_prob[:,0]
FCNClassifier_predictions_prob_df[FCNClassifier_grid_search.classes_[1]] = FCNClassifier_predictions_prob[:,1] 


# Generate Model Metrics
FCNClassifier_accuracy = accuracy_score(y_test, FCNClassifier_predictions.iloc[:,0])
FCNClassifier_f1_score = f1_score(y_test, FCNClassifier_predictions.iloc[:,0],pos_label='2')
FCNClassifier_precision = precision_score(y_test, FCNClassifier_predictions.iloc[:,0],pos_label='2')
FCNClassifier_recall = recall_score(y_test, FCNClassifier_predictions.iloc[:,0],pos_label='2')
FCNClassifier_roc_auc_score = roc_auc_score(y_test, FCNClassifier_predictions_prob_df[FCNClassifier_grid_search.classes_[1]])
FCNClassifier_performance_metrics = [['FCNClassifier','accuracy',FCNClassifier_accuracy], 
                                  ['FCNClassifier','f1_score',FCNClassifier_f1_score],
                                  ['FCNClassifier','precision', FCNClassifier_precision],
                                  ['FCNClassifier','recall', FCNClassifier_recall],
                                  ['FCNClassifier','roc_auc_score', FCNClassifier_roc_auc_score]]
FCNClassifier_performance_metrics = pd.DataFrame(FCNClassifier_performance_metrics, columns=['model','metric', 'value'])
fpr, tpr, thresholds = roc_curve(y_test, FCNClassifier_predictions_prob_df[FCNClassifier_grid_search.classes_[1]],pos_label='2')
roc_auc = auc(fpr, tpr)

# ROC Curve plot
FCNClassifier_roc_auc_plot, FCNClassifier_roc_auc_plot_ax = plt.subplots()
FCNClassifier_roc_auc_plot_ax.plot(fpr, tpr, label=f'ROC curve (AUC = {roc_auc:.4f})')
FCNClassifier_roc_auc_plot_ax.plot([0, 1], [0, 1], 'r--', label='Random guess')
# Set axis labels and title
FCNClassifier_roc_auc_plot_ax.set_xlabel('False Positive Rate')
FCNClassifier_roc_auc_plot_ax.set_ylabel('True Positive Rate')
FCNClassifier_roc_auc_plot_ax.set_title(f'FCNClassifier ROC Curve')
# Add legend
FCNClassifier_roc_auc_plot_ax.legend()


print(FCNClassifier_performance_metrics[FCNClassifier_performance_metrics['metric'] == 'roc_auc_score'])
model_comparison_list.append(FCNClassifier_performance_metrics)##### End of Model Pipeline for FCN #####
##### Model Pipeline for INCEPTIONTIME #####

from sktime.classification.deep_learning.inceptiontime import InceptionTimeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score,make_scorer,f1_score,precision_score,recall_score,roc_auc_score,roc_curve,auc
import matplotlib.pyplot as plt
InceptionTimeClassifier_param_grid = {
"n_epochs": np.arange(100, 2000, 1000),
}

InceptionTimeClassifier_model = InceptionTimeClassifier()

# Create the grid search
InceptionTimeClassifier_grid_search = GridSearchCV(estimator=InceptionTimeClassifier_model, param_grid=InceptionTimeClassifier_param_grid, cv=3, scoring=make_scorer(accuracy_score), verbose=3)
InceptionTimeClassifier_grid_search.fit(X_train, y_train)

# Get the best hyperparameters
InceptionTimeClassifier_best_estimator = InceptionTimeClassifier_grid_search.best_estimator_

# Store results as a dataframe  
InceptionTimeClassifier_search_results = pd.DataFrame(InceptionTimeClassifier_grid_search.cv_results_)

# Generate Predictions
InceptionTimeClassifier_predictions = pd.DataFrame(InceptionTimeClassifier_best_estimator.predict(X_test))
InceptionTimeClassifier_predictions_prob = InceptionTimeClassifier_best_estimator.predict_proba(X_test)
InceptionTimeClassifier_predictions_prob_df = pd.DataFrame()
InceptionTimeClassifier_predictions_prob_df[InceptionTimeClassifier_grid_search.classes_[0]] = InceptionTimeClassifier_predictions_prob[:,0]
InceptionTimeClassifier_predictions_prob_df[InceptionTimeClassifier_grid_search.classes_[1]] = InceptionTimeClassifier_predictions_prob[:,1] 


# Generate Model Metrics
InceptionTimeClassifier_accuracy = accuracy_score(y_test, InceptionTimeClassifier_predictions.iloc[:,0])
InceptionTimeClassifier_f1_score = f1_score(y_test, InceptionTimeClassifier_predictions.iloc[:,0],pos_label='2')
InceptionTimeClassifier_precision = precision_score(y_test, InceptionTimeClassifier_predictions.iloc[:,0],pos_label='2')
InceptionTimeClassifier_recall = recall_score(y_test, InceptionTimeClassifier_predictions.iloc[:,0],pos_label='2')
InceptionTimeClassifier_roc_auc_score = roc_auc_score(y_test, InceptionTimeClassifier_predictions_prob_df[InceptionTimeClassifier_grid_search.classes_[1]])
InceptionTimeClassifier_performance_metrics = [['InceptionTimeClassifier','accuracy',InceptionTimeClassifier_accuracy], 
                                  ['InceptionTimeClassifier','f1_score',InceptionTimeClassifier_f1_score],
                                  ['InceptionTimeClassifier','precision', InceptionTimeClassifier_precision],
                                  ['InceptionTimeClassifier','recall', InceptionTimeClassifier_recall],
                                  ['InceptionTimeClassifier','roc_auc_score', InceptionTimeClassifier_roc_auc_score]]
InceptionTimeClassifier_performance_metrics = pd.DataFrame(InceptionTimeClassifier_performance_metrics, columns=['model','metric', 'value'])
fpr, tpr, thresholds = roc_curve(y_test, InceptionTimeClassifier_predictions_prob_df[InceptionTimeClassifier_grid_search.classes_[1]],pos_label='2')
roc_auc = auc(fpr, tpr)

# ROC Curve plot
InceptionTimeClassifier_roc_auc_plot, InceptionTimeClassifier_roc_auc_plot_ax = plt.subplots()
InceptionTimeClassifier_roc_auc_plot_ax.plot(fpr, tpr, label=f'ROC curve (AUC = {roc_auc:.4f})')
InceptionTimeClassifier_roc_auc_plot_ax.plot([0, 1], [0, 1], 'r--', label='Random guess')
# Set axis labels and title
InceptionTimeClassifier_roc_auc_plot_ax.set_xlabel('False Positive Rate')
InceptionTimeClassifier_roc_auc_plot_ax.set_ylabel('True Positive Rate')
InceptionTimeClassifier_roc_auc_plot_ax.set_title(f'InceptionTimeClassifier ROC Curve')
# Add legend
InceptionTimeClassifier_roc_auc_plot_ax.legend()


print(InceptionTimeClassifier_performance_metrics[InceptionTimeClassifier_performance_metrics['metric'] == 'roc_auc_score'])
model_comparison_list.append(InceptionTimeClassifier_performance_metrics)##### End of Model Pipeline for INCEPTIONTIME #####
##### Model Pipeline for MLP #####

from sktime.classification.deep_learning.mlp import MLPClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score,make_scorer,f1_score,precision_score,recall_score,roc_auc_score,roc_curve,auc
import matplotlib.pyplot as plt
MLPClassifier_param_grid = {
"n_epochs": np.arange(100, 2000, 1000),
"activation": ['sigmoid'],
}

MLPClassifier_model = MLPClassifier()

# Create the grid search
MLPClassifier_grid_search = GridSearchCV(estimator=MLPClassifier_model, param_grid=MLPClassifier_param_grid, cv=3, scoring=make_scorer(accuracy_score), verbose=3)
MLPClassifier_grid_search.fit(X_train, y_train)

# Get the best hyperparameters
MLPClassifier_best_estimator = MLPClassifier_grid_search.best_estimator_

# Store results as a dataframe  
MLPClassifier_search_results = pd.DataFrame(MLPClassifier_grid_search.cv_results_)

# Generate Predictions
MLPClassifier_predictions = pd.DataFrame(MLPClassifier_best_estimator.predict(X_test))
MLPClassifier_predictions_prob = MLPClassifier_best_estimator.predict_proba(X_test)
MLPClassifier_predictions_prob_df = pd.DataFrame()
MLPClassifier_predictions_prob_df[MLPClassifier_grid_search.classes_[0]] = MLPClassifier_predictions_prob[:,0]
MLPClassifier_predictions_prob_df[MLPClassifier_grid_search.classes_[1]] = MLPClassifier_predictions_prob[:,1] 


# Generate Model Metrics
MLPClassifier_accuracy = accuracy_score(y_test, MLPClassifier_predictions.iloc[:,0])
MLPClassifier_f1_score = f1_score(y_test, MLPClassifier_predictions.iloc[:,0],pos_label='2')
MLPClassifier_precision = precision_score(y_test, MLPClassifier_predictions.iloc[:,0],pos_label='2')
MLPClassifier_recall = recall_score(y_test, MLPClassifier_predictions.iloc[:,0],pos_label='2')
MLPClassifier_roc_auc_score = roc_auc_score(y_test, MLPClassifier_predictions_prob_df[MLPClassifier_grid_search.classes_[1]])
MLPClassifier_performance_metrics = [['MLPClassifier','accuracy',MLPClassifier_accuracy], 
                                  ['MLPClassifier','f1_score',MLPClassifier_f1_score],
                                  ['MLPClassifier','precision', MLPClassifier_precision],
                                  ['MLPClassifier','recall', MLPClassifier_recall],
                                  ['MLPClassifier','roc_auc_score', MLPClassifier_roc_auc_score]]
MLPClassifier_performance_metrics = pd.DataFrame(MLPClassifier_performance_metrics, columns=['model','metric', 'value'])
fpr, tpr, thresholds = roc_curve(y_test, MLPClassifier_predictions_prob_df[MLPClassifier_grid_search.classes_[1]],pos_label='2')
roc_auc = auc(fpr, tpr)

# ROC Curve plot
MLPClassifier_roc_auc_plot, MLPClassifier_roc_auc_plot_ax = plt.subplots()
MLPClassifier_roc_auc_plot_ax.plot(fpr, tpr, label=f'ROC curve (AUC = {roc_auc:.4f})')
MLPClassifier_roc_auc_plot_ax.plot([0, 1], [0, 1], 'r--', label='Random guess')
# Set axis labels and title
MLPClassifier_roc_auc_plot_ax.set_xlabel('False Positive Rate')
MLPClassifier_roc_auc_plot_ax.set_ylabel('True Positive Rate')
MLPClassifier_roc_auc_plot_ax.set_title(f'MLPClassifier ROC Curve')
# Add legend
MLPClassifier_roc_auc_plot_ax.legend()


print(MLPClassifier_performance_metrics[MLPClassifier_performance_metrics['metric'] == 'roc_auc_score'])
model_comparison_list.append(MLPClassifier_performance_metrics)##### End of Model Pipeline for MLP #####
##### Model Pipeline for TAPNET #####

from sktime.classification.deep_learning.tapnet import TapNetClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score,make_scorer,f1_score,precision_score,recall_score,roc_auc_score,roc_curve,auc
import matplotlib.pyplot as plt
TapNetClassifier_param_grid = {
"n_epochs": np.arange(100, 2000, 1000),
"use_lstm": ['True'],
"use_cnn": ['True'],
"use_rp": ['True'],
"use_att": ['True'],
}

TapNetClassifier_model = TapNetClassifier()

# Create the grid search
TapNetClassifier_grid_search = GridSearchCV(estimator=TapNetClassifier_model, param_grid=TapNetClassifier_param_grid, cv=3, scoring=make_scorer(accuracy_score), verbose=3)
TapNetClassifier_grid_search.fit(X_train, y_train)

# Get the best hyperparameters
TapNetClassifier_best_estimator = TapNetClassifier_grid_search.best_estimator_

# Store results as a dataframe  
TapNetClassifier_search_results = pd.DataFrame(TapNetClassifier_grid_search.cv_results_)

# Generate Predictions
TapNetClassifier_predictions = pd.DataFrame(TapNetClassifier_best_estimator.predict(X_test))
TapNetClassifier_predictions_prob = TapNetClassifier_best_estimator.predict_proba(X_test)
TapNetClassifier_predictions_prob_df = pd.DataFrame()
TapNetClassifier_predictions_prob_df[TapNetClassifier_grid_search.classes_[0]] = TapNetClassifier_predictions_prob[:,0]
TapNetClassifier_predictions_prob_df[TapNetClassifier_grid_search.classes_[1]] = TapNetClassifier_predictions_prob[:,1] 


# Generate Model Metrics
TapNetClassifier_accuracy = accuracy_score(y_test, TapNetClassifier_predictions.iloc[:,0])
TapNetClassifier_f1_score = f1_score(y_test, TapNetClassifier_predictions.iloc[:,0],pos_label='2')
TapNetClassifier_precision = precision_score(y_test, TapNetClassifier_predictions.iloc[:,0],pos_label='2')
TapNetClassifier_recall = recall_score(y_test, TapNetClassifier_predictions.iloc[:,0],pos_label='2')
TapNetClassifier_roc_auc_score = roc_auc_score(y_test, TapNetClassifier_predictions_prob_df[TapNetClassifier_grid_search.classes_[1]])
TapNetClassifier_performance_metrics = [['TapNetClassifier','accuracy',TapNetClassifier_accuracy], 
                                  ['TapNetClassifier','f1_score',TapNetClassifier_f1_score],
                                  ['TapNetClassifier','precision', TapNetClassifier_precision],
                                  ['TapNetClassifier','recall', TapNetClassifier_recall],
                                  ['TapNetClassifier','roc_auc_score', TapNetClassifier_roc_auc_score]]
TapNetClassifier_performance_metrics = pd.DataFrame(TapNetClassifier_performance_metrics, columns=['model','metric', 'value'])
fpr, tpr, thresholds = roc_curve(y_test, TapNetClassifier_predictions_prob_df[TapNetClassifier_grid_search.classes_[1]],pos_label='2')
roc_auc = auc(fpr, tpr)

# ROC Curve plot
TapNetClassifier_roc_auc_plot, TapNetClassifier_roc_auc_plot_ax = plt.subplots()
TapNetClassifier_roc_auc_plot_ax.plot(fpr, tpr, label=f'ROC curve (AUC = {roc_auc:.4f})')
TapNetClassifier_roc_auc_plot_ax.plot([0, 1], [0, 1], 'r--', label='Random guess')
# Set axis labels and title
TapNetClassifier_roc_auc_plot_ax.set_xlabel('False Positive Rate')
TapNetClassifier_roc_auc_plot_ax.set_ylabel('True Positive Rate')
TapNetClassifier_roc_auc_plot_ax.set_title(f'TapNetClassifier ROC Curve')
# Add legend
TapNetClassifier_roc_auc_plot_ax.legend()


print(TapNetClassifier_performance_metrics[TapNetClassifier_performance_metrics['metric'] == 'roc_auc_score'])
model_comparison_list.append(TapNetClassifier_performance_metrics)##### End of Model Pipeline for TAPNET #####
##### Model Pipeline for BOSSENSEMBLE #####

from sktime.classification.dictionary_based import BOSSEnsemble
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score,make_scorer,f1_score,precision_score,recall_score,roc_auc_score,roc_curve,auc
import matplotlib.pyplot as plt
BOSSEnsemble_param_grid = {
"max_ensemble_size": np.arange(100, 2000, 1000),
"use_boss_distance": ['True'],
"feature_selection": ['none'],
}

BOSSEnsemble_model = BOSSEnsemble()

# Create the grid search
BOSSEnsemble_grid_search = GridSearchCV(estimator=BOSSEnsemble_model, param_grid=BOSSEnsemble_param_grid, cv=3, scoring=make_scorer(accuracy_score), verbose=3)
BOSSEnsemble_grid_search.fit(X_train, y_train)

# Get the best hyperparameters
BOSSEnsemble_best_estimator = BOSSEnsemble_grid_search.best_estimator_

# Store results as a dataframe  
BOSSEnsemble_search_results = pd.DataFrame(BOSSEnsemble_grid_search.cv_results_)

# Generate Predictions
BOSSEnsemble_predictions = pd.DataFrame(BOSSEnsemble_best_estimator.predict(X_test))
BOSSEnsemble_predictions_prob = BOSSEnsemble_best_estimator.predict_proba(X_test)
BOSSEnsemble_predictions_prob_df = pd.DataFrame()
BOSSEnsemble_predictions_prob_df[BOSSEnsemble_grid_search.classes_[0]] = BOSSEnsemble_predictions_prob[:,0]
BOSSEnsemble_predictions_prob_df[BOSSEnsemble_grid_search.classes_[1]] = BOSSEnsemble_predictions_prob[:,1] 


# Generate Model Metrics
BOSSEnsemble_accuracy = accuracy_score(y_test, BOSSEnsemble_predictions.iloc[:,0])
BOSSEnsemble_f1_score = f1_score(y_test, BOSSEnsemble_predictions.iloc[:,0],pos_label='2')
BOSSEnsemble_precision = precision_score(y_test, BOSSEnsemble_predictions.iloc[:,0],pos_label='2')
BOSSEnsemble_recall = recall_score(y_test, BOSSEnsemble_predictions.iloc[:,0],pos_label='2')
BOSSEnsemble_roc_auc_score = roc_auc_score(y_test, BOSSEnsemble_predictions_prob_df[BOSSEnsemble_grid_search.classes_[1]])
BOSSEnsemble_performance_metrics = [['BOSSEnsemble','accuracy',BOSSEnsemble_accuracy], 
                                  ['BOSSEnsemble','f1_score',BOSSEnsemble_f1_score],
                                  ['BOSSEnsemble','precision', BOSSEnsemble_precision],
                                  ['BOSSEnsemble','recall', BOSSEnsemble_recall],
                                  ['BOSSEnsemble','roc_auc_score', BOSSEnsemble_roc_auc_score]]
BOSSEnsemble_performance_metrics = pd.DataFrame(BOSSEnsemble_performance_metrics, columns=['model','metric', 'value'])
fpr, tpr, thresholds = roc_curve(y_test, BOSSEnsemble_predictions_prob_df[BOSSEnsemble_grid_search.classes_[1]],pos_label='2')
roc_auc = auc(fpr, tpr)

# ROC Curve plot
BOSSEnsemble_roc_auc_plot, BOSSEnsemble_roc_auc_plot_ax = plt.subplots()
BOSSEnsemble_roc_auc_plot_ax.plot(fpr, tpr, label=f'ROC curve (AUC = {roc_auc:.4f})')
BOSSEnsemble_roc_auc_plot_ax.plot([0, 1], [0, 1], 'r--', label='Random guess')
# Set axis labels and title
BOSSEnsemble_roc_auc_plot_ax.set_xlabel('False Positive Rate')
BOSSEnsemble_roc_auc_plot_ax.set_ylabel('True Positive Rate')
BOSSEnsemble_roc_auc_plot_ax.set_title(f'BOSSEnsemble ROC Curve')
# Add legend
BOSSEnsemble_roc_auc_plot_ax.legend()


print(BOSSEnsemble_performance_metrics[BOSSEnsemble_performance_metrics['metric'] == 'roc_auc_score'])
model_comparison_list.append(BOSSEnsemble_performance_metrics)##### End of Model Pipeline for BOSSENSEMBLE #####
##### Model Pipeline for CONTRACTABLEBOSS #####

from sktime.classification.dictionary_based import ContractableBOSS
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score,make_scorer,f1_score,precision_score,recall_score,roc_auc_score,roc_curve,auc
import matplotlib.pyplot as plt
ContractableBOSS_param_grid = {
"max_ensemble_size": np.arange(100, 2000, 1000),
"feature_selection": ['none'],
}

ContractableBOSS_model = ContractableBOSS()

# Create the grid search
ContractableBOSS_grid_search = GridSearchCV(estimator=ContractableBOSS_model, param_grid=ContractableBOSS_param_grid, cv=3, scoring=make_scorer(accuracy_score), verbose=3)
ContractableBOSS_grid_search.fit(X_train, y_train)

# Get the best hyperparameters
ContractableBOSS_best_estimator = ContractableBOSS_grid_search.best_estimator_

# Store results as a dataframe  
ContractableBOSS_search_results = pd.DataFrame(ContractableBOSS_grid_search.cv_results_)

# Generate Predictions
ContractableBOSS_predictions = pd.DataFrame(ContractableBOSS_best_estimator.predict(X_test))
ContractableBOSS_predictions_prob = ContractableBOSS_best_estimator.predict_proba(X_test)
ContractableBOSS_predictions_prob_df = pd.DataFrame()
ContractableBOSS_predictions_prob_df[ContractableBOSS_grid_search.classes_[0]] = ContractableBOSS_predictions_prob[:,0]
ContractableBOSS_predictions_prob_df[ContractableBOSS_grid_search.classes_[1]] = ContractableBOSS_predictions_prob[:,1] 


# Generate Model Metrics
ContractableBOSS_accuracy = accuracy_score(y_test, ContractableBOSS_predictions.iloc[:,0])
ContractableBOSS_f1_score = f1_score(y_test, ContractableBOSS_predictions.iloc[:,0],pos_label='2')
ContractableBOSS_precision = precision_score(y_test, ContractableBOSS_predictions.iloc[:,0],pos_label='2')
ContractableBOSS_recall = recall_score(y_test, ContractableBOSS_predictions.iloc[:,0],pos_label='2')
ContractableBOSS_roc_auc_score = roc_auc_score(y_test, ContractableBOSS_predictions_prob_df[ContractableBOSS_grid_search.classes_[1]])
ContractableBOSS_performance_metrics = [['ContractableBOSS','accuracy',ContractableBOSS_accuracy], 
                                  ['ContractableBOSS','f1_score',ContractableBOSS_f1_score],
                                  ['ContractableBOSS','precision', ContractableBOSS_precision],
                                  ['ContractableBOSS','recall', ContractableBOSS_recall],
                                  ['ContractableBOSS','roc_auc_score', ContractableBOSS_roc_auc_score]]
ContractableBOSS_performance_metrics = pd.DataFrame(ContractableBOSS_performance_metrics, columns=['model','metric', 'value'])
fpr, tpr, thresholds = roc_curve(y_test, ContractableBOSS_predictions_prob_df[ContractableBOSS_grid_search.classes_[1]],pos_label='2')
roc_auc = auc(fpr, tpr)

# ROC Curve plot
ContractableBOSS_roc_auc_plot, ContractableBOSS_roc_auc_plot_ax = plt.subplots()
ContractableBOSS_roc_auc_plot_ax.plot(fpr, tpr, label=f'ROC curve (AUC = {roc_auc:.4f})')
ContractableBOSS_roc_auc_plot_ax.plot([0, 1], [0, 1], 'r--', label='Random guess')
# Set axis labels and title
ContractableBOSS_roc_auc_plot_ax.set_xlabel('False Positive Rate')
ContractableBOSS_roc_auc_plot_ax.set_ylabel('True Positive Rate')
ContractableBOSS_roc_auc_plot_ax.set_title(f'ContractableBOSS ROC Curve')
# Add legend
ContractableBOSS_roc_auc_plot_ax.legend()


print(ContractableBOSS_performance_metrics[ContractableBOSS_performance_metrics['metric'] == 'roc_auc_score'])
model_comparison_list.append(ContractableBOSS_performance_metrics)##### End of Model Pipeline for CONTRACTABLEBOSS #####
##### Model Pipeline for ElasticEnsemble #####

from sktime.classification.distance_based import ElasticEnsemble
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score,make_scorer,f1_score,precision_score,recall_score,roc_auc_score,roc_curve,auc
import matplotlib.pyplot as plt
ElasticEnsemble_param_grid = {
"proportion_of_param_options": np.arange(0.1, 1.0, 0.5),
"distance_measures": ['euclidean'],
}

ElasticEnsemble_model = ElasticEnsemble()

# Create the grid search
ElasticEnsemble_grid_search = GridSearchCV(estimator=ElasticEnsemble_model, param_grid=ElasticEnsemble_param_grid, cv=3, scoring=make_scorer(accuracy_score), verbose=3)
ElasticEnsemble_grid_search.fit(X_train, y_train)

# Get the best hyperparameters
ElasticEnsemble_best_estimator = ElasticEnsemble_grid_search.best_estimator_

# Store results as a dataframe  
ElasticEnsemble_search_results = pd.DataFrame(ElasticEnsemble_grid_search.cv_results_)

# Generate Predictions
ElasticEnsemble_predictions = pd.DataFrame(ElasticEnsemble_best_estimator.predict(X_test))
ElasticEnsemble_predictions_prob = ElasticEnsemble_best_estimator.predict_proba(X_test)
ElasticEnsemble_predictions_prob_df = pd.DataFrame()
ElasticEnsemble_predictions_prob_df[ElasticEnsemble_grid_search.classes_[0]] = ElasticEnsemble_predictions_prob[:,0]
ElasticEnsemble_predictions_prob_df[ElasticEnsemble_grid_search.classes_[1]] = ElasticEnsemble_predictions_prob[:,1] 


# Generate Model Metrics
ElasticEnsemble_accuracy = accuracy_score(y_test, ElasticEnsemble_predictions.iloc[:,0])
ElasticEnsemble_f1_score = f1_score(y_test, ElasticEnsemble_predictions.iloc[:,0],pos_label='2')
ElasticEnsemble_precision = precision_score(y_test, ElasticEnsemble_predictions.iloc[:,0],pos_label='2')
ElasticEnsemble_recall = recall_score(y_test, ElasticEnsemble_predictions.iloc[:,0],pos_label='2')
ElasticEnsemble_roc_auc_score = roc_auc_score(y_test, ElasticEnsemble_predictions_prob_df[ElasticEnsemble_grid_search.classes_[1]])
ElasticEnsemble_performance_metrics = [['ElasticEnsemble','accuracy',ElasticEnsemble_accuracy], 
                                  ['ElasticEnsemble','f1_score',ElasticEnsemble_f1_score],
                                  ['ElasticEnsemble','precision', ElasticEnsemble_precision],
                                  ['ElasticEnsemble','recall', ElasticEnsemble_recall],
                                  ['ElasticEnsemble','roc_auc_score', ElasticEnsemble_roc_auc_score]]
ElasticEnsemble_performance_metrics = pd.DataFrame(ElasticEnsemble_performance_metrics, columns=['model','metric', 'value'])
fpr, tpr, thresholds = roc_curve(y_test, ElasticEnsemble_predictions_prob_df[ElasticEnsemble_grid_search.classes_[1]],pos_label='2')
roc_auc = auc(fpr, tpr)

# ROC Curve plot
ElasticEnsemble_roc_auc_plot, ElasticEnsemble_roc_auc_plot_ax = plt.subplots()
ElasticEnsemble_roc_auc_plot_ax.plot(fpr, tpr, label=f'ROC curve (AUC = {roc_auc:.4f})')
ElasticEnsemble_roc_auc_plot_ax.plot([0, 1], [0, 1], 'r--', label='Random guess')
# Set axis labels and title
ElasticEnsemble_roc_auc_plot_ax.set_xlabel('False Positive Rate')
ElasticEnsemble_roc_auc_plot_ax.set_ylabel('True Positive Rate')
ElasticEnsemble_roc_auc_plot_ax.set_title(f'ElasticEnsemble ROC Curve')
# Add legend
ElasticEnsemble_roc_auc_plot_ax.legend()


print(ElasticEnsemble_performance_metrics[ElasticEnsemble_performance_metrics['metric'] == 'roc_auc_score'])
model_comparison_list.append(ElasticEnsemble_performance_metrics)##### End of Model Pipeline for ElasticEnsemble #####
##### Model Pipeline for KNN #####

from sktime.classification.distance_based import KNeighborsTimeSeriesClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score,make_scorer,f1_score,precision_score,recall_score,roc_auc_score,roc_curve,auc
import matplotlib.pyplot as plt
KNeighborsTimeSeriesClassifier_param_grid = {
"n_neighbors": np.arange(1, 10, 2),
"distance": ['euclidean'],
"weights": ['uniform'],
"algorithm": ['brute'],
}

KNeighborsTimeSeriesClassifier_model = KNeighborsTimeSeriesClassifier()

# Create the grid search
KNeighborsTimeSeriesClassifier_grid_search = GridSearchCV(estimator=KNeighborsTimeSeriesClassifier_model, param_grid=KNeighborsTimeSeriesClassifier_param_grid, cv=3, scoring=make_scorer(accuracy_score), verbose=3)
KNeighborsTimeSeriesClassifier_grid_search.fit(X_train, y_train)

# Get the best hyperparameters
KNeighborsTimeSeriesClassifier_best_estimator = KNeighborsTimeSeriesClassifier_grid_search.best_estimator_

# Store results as a dataframe  
KNeighborsTimeSeriesClassifier_search_results = pd.DataFrame(KNeighborsTimeSeriesClassifier_grid_search.cv_results_)

# Generate Predictions
KNeighborsTimeSeriesClassifier_predictions = pd.DataFrame(KNeighborsTimeSeriesClassifier_best_estimator.predict(X_test))
KNeighborsTimeSeriesClassifier_predictions_prob = KNeighborsTimeSeriesClassifier_best_estimator.predict_proba(X_test)
KNeighborsTimeSeriesClassifier_predictions_prob_df = pd.DataFrame()
KNeighborsTimeSeriesClassifier_predictions_prob_df[KNeighborsTimeSeriesClassifier_grid_search.classes_[0]] = KNeighborsTimeSeriesClassifier_predictions_prob[:,0]
KNeighborsTimeSeriesClassifier_predictions_prob_df[KNeighborsTimeSeriesClassifier_grid_search.classes_[1]] = KNeighborsTimeSeriesClassifier_predictions_prob[:,1] 


# Generate Model Metrics
KNeighborsTimeSeriesClassifier_accuracy = accuracy_score(y_test, KNeighborsTimeSeriesClassifier_predictions.iloc[:,0])
KNeighborsTimeSeriesClassifier_f1_score = f1_score(y_test, KNeighborsTimeSeriesClassifier_predictions.iloc[:,0],pos_label='2')
KNeighborsTimeSeriesClassifier_precision = precision_score(y_test, KNeighborsTimeSeriesClassifier_predictions.iloc[:,0],pos_label='2')
KNeighborsTimeSeriesClassifier_recall = recall_score(y_test, KNeighborsTimeSeriesClassifier_predictions.iloc[:,0],pos_label='2')
KNeighborsTimeSeriesClassifier_roc_auc_score = roc_auc_score(y_test, KNeighborsTimeSeriesClassifier_predictions_prob_df[KNeighborsTimeSeriesClassifier_grid_search.classes_[1]])
KNeighborsTimeSeriesClassifier_performance_metrics = [['KNeighborsTimeSeriesClassifier','accuracy',KNeighborsTimeSeriesClassifier_accuracy], 
                                  ['KNeighborsTimeSeriesClassifier','f1_score',KNeighborsTimeSeriesClassifier_f1_score],
                                  ['KNeighborsTimeSeriesClassifier','precision', KNeighborsTimeSeriesClassifier_precision],
                                  ['KNeighborsTimeSeriesClassifier','recall', KNeighborsTimeSeriesClassifier_recall],
                                  ['KNeighborsTimeSeriesClassifier','roc_auc_score', KNeighborsTimeSeriesClassifier_roc_auc_score]]
KNeighborsTimeSeriesClassifier_performance_metrics = pd.DataFrame(KNeighborsTimeSeriesClassifier_performance_metrics, columns=['model','metric', 'value'])
fpr, tpr, thresholds = roc_curve(y_test, KNeighborsTimeSeriesClassifier_predictions_prob_df[KNeighborsTimeSeriesClassifier_grid_search.classes_[1]],pos_label='2')
roc_auc = auc(fpr, tpr)

# ROC Curve plot
KNeighborsTimeSeriesClassifier_roc_auc_plot, KNeighborsTimeSeriesClassifier_roc_auc_plot_ax = plt.subplots()
KNeighborsTimeSeriesClassifier_roc_auc_plot_ax.plot(fpr, tpr, label=f'ROC curve (AUC = {roc_auc:.4f})')
KNeighborsTimeSeriesClassifier_roc_auc_plot_ax.plot([0, 1], [0, 1], 'r--', label='Random guess')
# Set axis labels and title
KNeighborsTimeSeriesClassifier_roc_auc_plot_ax.set_xlabel('False Positive Rate')
KNeighborsTimeSeriesClassifier_roc_auc_plot_ax.set_ylabel('True Positive Rate')
KNeighborsTimeSeriesClassifier_roc_auc_plot_ax.set_title(f'KNeighborsTimeSeriesClassifier ROC Curve')
# Add legend
KNeighborsTimeSeriesClassifier_roc_auc_plot_ax.legend()


print(KNeighborsTimeSeriesClassifier_performance_metrics[KNeighborsTimeSeriesClassifier_performance_metrics['metric'] == 'roc_auc_score'])
model_comparison_list.append(KNeighborsTimeSeriesClassifier_performance_metrics)##### End of Model Pipeline for KNN #####
##### Model Pipeline for ProximityForest #####

from sktime.classification.distance_based import ProximityForest
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score,make_scorer,f1_score,precision_score,recall_score,roc_auc_score,roc_curve,auc
import matplotlib.pyplot as plt
ProximityForest_param_grid = {
"n_estimators": np.arange(10, 100, 20),
"max_depth": np.arange(10, 20, 5),
}

ProximityForest_model = ProximityForest()

# Create the grid search
ProximityForest_grid_search = GridSearchCV(estimator=ProximityForest_model, param_grid=ProximityForest_param_grid, cv=3, scoring=make_scorer(accuracy_score), verbose=3)
ProximityForest_grid_search.fit(X_train, y_train)

# Get the best hyperparameters
ProximityForest_best_estimator = ProximityForest_grid_search.best_estimator_

# Store results as a dataframe  
ProximityForest_search_results = pd.DataFrame(ProximityForest_grid_search.cv_results_)

# Generate Predictions
ProximityForest_predictions = pd.DataFrame(ProximityForest_best_estimator.predict(X_test))
ProximityForest_predictions_prob = ProximityForest_best_estimator.predict_proba(X_test)
ProximityForest_predictions_prob_df = pd.DataFrame()
ProximityForest_predictions_prob_df[ProximityForest_grid_search.classes_[0]] = ProximityForest_predictions_prob[:,0]
ProximityForest_predictions_prob_df[ProximityForest_grid_search.classes_[1]] = ProximityForest_predictions_prob[:,1] 


# Generate Model Metrics
ProximityForest_accuracy = accuracy_score(y_test, ProximityForest_predictions.iloc[:,0])
ProximityForest_f1_score = f1_score(y_test, ProximityForest_predictions.iloc[:,0],pos_label='2')
ProximityForest_precision = precision_score(y_test, ProximityForest_predictions.iloc[:,0],pos_label='2')
ProximityForest_recall = recall_score(y_test, ProximityForest_predictions.iloc[:,0],pos_label='2')
ProximityForest_roc_auc_score = roc_auc_score(y_test, ProximityForest_predictions_prob_df[ProximityForest_grid_search.classes_[1]])
ProximityForest_performance_metrics = [['ProximityForest','accuracy',ProximityForest_accuracy], 
                                  ['ProximityForest','f1_score',ProximityForest_f1_score],
                                  ['ProximityForest','precision', ProximityForest_precision],
                                  ['ProximityForest','recall', ProximityForest_recall],
                                  ['ProximityForest','roc_auc_score', ProximityForest_roc_auc_score]]
ProximityForest_performance_metrics = pd.DataFrame(ProximityForest_performance_metrics, columns=['model','metric', 'value'])
fpr, tpr, thresholds = roc_curve(y_test, ProximityForest_predictions_prob_df[ProximityForest_grid_search.classes_[1]],pos_label='2')
roc_auc = auc(fpr, tpr)

# ROC Curve plot
ProximityForest_roc_auc_plot, ProximityForest_roc_auc_plot_ax = plt.subplots()
ProximityForest_roc_auc_plot_ax.plot(fpr, tpr, label=f'ROC curve (AUC = {roc_auc:.4f})')
ProximityForest_roc_auc_plot_ax.plot([0, 1], [0, 1], 'r--', label='Random guess')
# Set axis labels and title
ProximityForest_roc_auc_plot_ax.set_xlabel('False Positive Rate')
ProximityForest_roc_auc_plot_ax.set_ylabel('True Positive Rate')
ProximityForest_roc_auc_plot_ax.set_title(f'ProximityForest ROC Curve')
# Add legend
ProximityForest_roc_auc_plot_ax.legend()


print(ProximityForest_performance_metrics[ProximityForest_performance_metrics['metric'] == 'roc_auc_score'])
model_comparison_list.append(ProximityForest_performance_metrics)##### End of Model Pipeline for ProximityForest #####
##### Model Pipeline for ProximityStump #####

from sktime.classification.distance_based import ProximityStump
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score,make_scorer,f1_score,precision_score,recall_score,roc_auc_score,roc_curve,auc
import matplotlib.pyplot as plt
ProximityStump_param_grid = {
}

ProximityStump_model = ProximityStump()

# Create the grid search
ProximityStump_grid_search = GridSearchCV(estimator=ProximityStump_model, param_grid=ProximityStump_param_grid, cv=3, scoring=make_scorer(accuracy_score), verbose=3)
ProximityStump_grid_search.fit(X_train, y_train)

# Get the best hyperparameters
ProximityStump_best_estimator = ProximityStump_grid_search.best_estimator_

# Store results as a dataframe  
ProximityStump_search_results = pd.DataFrame(ProximityStump_grid_search.cv_results_)

# Generate Predictions
ProximityStump_predictions = pd.DataFrame(ProximityStump_best_estimator.predict(X_test))
ProximityStump_predictions_prob = ProximityStump_best_estimator.predict_proba(X_test)
ProximityStump_predictions_prob_df = pd.DataFrame()
ProximityStump_predictions_prob_df[ProximityStump_grid_search.classes_[0]] = ProximityStump_predictions_prob[:,0]
ProximityStump_predictions_prob_df[ProximityStump_grid_search.classes_[1]] = ProximityStump_predictions_prob[:,1] 


# Generate Model Metrics
ProximityStump_accuracy = accuracy_score(y_test, ProximityStump_predictions.iloc[:,0])
ProximityStump_f1_score = f1_score(y_test, ProximityStump_predictions.iloc[:,0],pos_label='2')
ProximityStump_precision = precision_score(y_test, ProximityStump_predictions.iloc[:,0],pos_label='2')
ProximityStump_recall = recall_score(y_test, ProximityStump_predictions.iloc[:,0],pos_label='2')
ProximityStump_roc_auc_score = roc_auc_score(y_test, ProximityStump_predictions_prob_df[ProximityStump_grid_search.classes_[1]])
ProximityStump_performance_metrics = [['ProximityStump','accuracy',ProximityStump_accuracy], 
                                  ['ProximityStump','f1_score',ProximityStump_f1_score],
                                  ['ProximityStump','precision', ProximityStump_precision],
                                  ['ProximityStump','recall', ProximityStump_recall],
                                  ['ProximityStump','roc_auc_score', ProximityStump_roc_auc_score]]
ProximityStump_performance_metrics = pd.DataFrame(ProximityStump_performance_metrics, columns=['model','metric', 'value'])
fpr, tpr, thresholds = roc_curve(y_test, ProximityStump_predictions_prob_df[ProximityStump_grid_search.classes_[1]],pos_label='2')
roc_auc = auc(fpr, tpr)

# ROC Curve plot
ProximityStump_roc_auc_plot, ProximityStump_roc_auc_plot_ax = plt.subplots()
ProximityStump_roc_auc_plot_ax.plot(fpr, tpr, label=f'ROC curve (AUC = {roc_auc:.4f})')
ProximityStump_roc_auc_plot_ax.plot([0, 1], [0, 1], 'r--', label='Random guess')
# Set axis labels and title
ProximityStump_roc_auc_plot_ax.set_xlabel('False Positive Rate')
ProximityStump_roc_auc_plot_ax.set_ylabel('True Positive Rate')
ProximityStump_roc_auc_plot_ax.set_title(f'ProximityStump ROC Curve')
# Add legend
ProximityStump_roc_auc_plot_ax.legend()


print(ProximityStump_performance_metrics[ProximityStump_performance_metrics['metric'] == 'roc_auc_score'])
model_comparison_list.append(ProximityStump_performance_metrics)##### End of Model Pipeline for ProximityStump #####
##### Model Pipeline for ProximityTree #####

from sktime.classification.distance_based import ProximityTree
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score,make_scorer,f1_score,precision_score,recall_score,roc_auc_score,roc_curve,auc
import matplotlib.pyplot as plt
ProximityTree_param_grid = {
}

ProximityTree_model = ProximityTree()

# Create the grid search
ProximityTree_grid_search = GridSearchCV(estimator=ProximityTree_model, param_grid=ProximityTree_param_grid, cv=3, scoring=make_scorer(accuracy_score), verbose=3)
ProximityTree_grid_search.fit(X_train, y_train)

# Get the best hyperparameters
ProximityTree_best_estimator = ProximityTree_grid_search.best_estimator_

# Store results as a dataframe  
ProximityTree_search_results = pd.DataFrame(ProximityTree_grid_search.cv_results_)

# Generate Predictions
ProximityTree_predictions = pd.DataFrame(ProximityTree_best_estimator.predict(X_test))
ProximityTree_predictions_prob = ProximityTree_best_estimator.predict_proba(X_test)
ProximityTree_predictions_prob_df = pd.DataFrame()
ProximityTree_predictions_prob_df[ProximityTree_grid_search.classes_[0]] = ProximityTree_predictions_prob[:,0]
ProximityTree_predictions_prob_df[ProximityTree_grid_search.classes_[1]] = ProximityTree_predictions_prob[:,1] 


# Generate Model Metrics
ProximityTree_accuracy = accuracy_score(y_test, ProximityTree_predictions.iloc[:,0])
ProximityTree_f1_score = f1_score(y_test, ProximityTree_predictions.iloc[:,0],pos_label='2')
ProximityTree_precision = precision_score(y_test, ProximityTree_predictions.iloc[:,0],pos_label='2')
ProximityTree_recall = recall_score(y_test, ProximityTree_predictions.iloc[:,0],pos_label='2')
ProximityTree_roc_auc_score = roc_auc_score(y_test, ProximityTree_predictions_prob_df[ProximityTree_grid_search.classes_[1]])
ProximityTree_performance_metrics = [['ProximityTree','accuracy',ProximityTree_accuracy], 
                                  ['ProximityTree','f1_score',ProximityTree_f1_score],
                                  ['ProximityTree','precision', ProximityTree_precision],
                                  ['ProximityTree','recall', ProximityTree_recall],
                                  ['ProximityTree','roc_auc_score', ProximityTree_roc_auc_score]]
ProximityTree_performance_metrics = pd.DataFrame(ProximityTree_performance_metrics, columns=['model','metric', 'value'])
fpr, tpr, thresholds = roc_curve(y_test, ProximityTree_predictions_prob_df[ProximityTree_grid_search.classes_[1]],pos_label='2')
roc_auc = auc(fpr, tpr)

# ROC Curve plot
ProximityTree_roc_auc_plot, ProximityTree_roc_auc_plot_ax = plt.subplots()
ProximityTree_roc_auc_plot_ax.plot(fpr, tpr, label=f'ROC curve (AUC = {roc_auc:.4f})')
ProximityTree_roc_auc_plot_ax.plot([0, 1], [0, 1], 'r--', label='Random guess')
# Set axis labels and title
ProximityTree_roc_auc_plot_ax.set_xlabel('False Positive Rate')
ProximityTree_roc_auc_plot_ax.set_ylabel('True Positive Rate')
ProximityTree_roc_auc_plot_ax.set_title(f'ProximityTree ROC Curve')
# Add legend
ProximityTree_roc_auc_plot_ax.legend()


print(ProximityTree_performance_metrics[ProximityTree_performance_metrics['metric'] == 'roc_auc_score'])
model_comparison_list.append(ProximityTree_performance_metrics)##### End of Model Pipeline for ProximityTree #####
##### Model Pipeline for ShapeDTW #####

from sktime.classification.distance_based import ShapeDTW
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score,make_scorer,f1_score,precision_score,recall_score,roc_auc_score,roc_curve,auc
import matplotlib.pyplot as plt
ShapeDTW_param_grid = {
"n_neighbors": np.arange(1, 10, 5),
"shape_descriptor_function": ['raw'],
}

ShapeDTW_model = ShapeDTW()

# Create the grid search
ShapeDTW_grid_search = GridSearchCV(estimator=ShapeDTW_model, param_grid=ShapeDTW_param_grid, cv=3, scoring=make_scorer(accuracy_score), verbose=3)
ShapeDTW_grid_search.fit(X_train, y_train)

# Get the best hyperparameters
ShapeDTW_best_estimator = ShapeDTW_grid_search.best_estimator_

# Store results as a dataframe  
ShapeDTW_search_results = pd.DataFrame(ShapeDTW_grid_search.cv_results_)

# Generate Predictions
ShapeDTW_predictions = pd.DataFrame(ShapeDTW_best_estimator.predict(X_test))
ShapeDTW_predictions_prob = ShapeDTW_best_estimator.predict_proba(X_test)
ShapeDTW_predictions_prob_df = pd.DataFrame()
ShapeDTW_predictions_prob_df[ShapeDTW_grid_search.classes_[0]] = ShapeDTW_predictions_prob[:,0]
ShapeDTW_predictions_prob_df[ShapeDTW_grid_search.classes_[1]] = ShapeDTW_predictions_prob[:,1] 


# Generate Model Metrics
ShapeDTW_accuracy = accuracy_score(y_test, ShapeDTW_predictions.iloc[:,0])
ShapeDTW_f1_score = f1_score(y_test, ShapeDTW_predictions.iloc[:,0],pos_label='2')
ShapeDTW_precision = precision_score(y_test, ShapeDTW_predictions.iloc[:,0],pos_label='2')
ShapeDTW_recall = recall_score(y_test, ShapeDTW_predictions.iloc[:,0],pos_label='2')
ShapeDTW_roc_auc_score = roc_auc_score(y_test, ShapeDTW_predictions_prob_df[ShapeDTW_grid_search.classes_[1]])
ShapeDTW_performance_metrics = [['ShapeDTW','accuracy',ShapeDTW_accuracy], 
                                  ['ShapeDTW','f1_score',ShapeDTW_f1_score],
                                  ['ShapeDTW','precision', ShapeDTW_precision],
                                  ['ShapeDTW','recall', ShapeDTW_recall],
                                  ['ShapeDTW','roc_auc_score', ShapeDTW_roc_auc_score]]
ShapeDTW_performance_metrics = pd.DataFrame(ShapeDTW_performance_metrics, columns=['model','metric', 'value'])
fpr, tpr, thresholds = roc_curve(y_test, ShapeDTW_predictions_prob_df[ShapeDTW_grid_search.classes_[1]],pos_label='2')
roc_auc = auc(fpr, tpr)

# ROC Curve plot
ShapeDTW_roc_auc_plot, ShapeDTW_roc_auc_plot_ax = plt.subplots()
ShapeDTW_roc_auc_plot_ax.plot(fpr, tpr, label=f'ROC curve (AUC = {roc_auc:.4f})')
ShapeDTW_roc_auc_plot_ax.plot([0, 1], [0, 1], 'r--', label='Random guess')
# Set axis labels and title
ShapeDTW_roc_auc_plot_ax.set_xlabel('False Positive Rate')
ShapeDTW_roc_auc_plot_ax.set_ylabel('True Positive Rate')
ShapeDTW_roc_auc_plot_ax.set_title(f'ShapeDTW ROC Curve')
# Add legend
ShapeDTW_roc_auc_plot_ax.legend()


print(ShapeDTW_performance_metrics[ShapeDTW_performance_metrics['metric'] == 'roc_auc_score'])
model_comparison_list.append(ShapeDTW_performance_metrics)##### End of Model Pipeline for ShapeDTW #####
##### Model Comparison #####
table = pd.concat(model_comparison_list)
table = table.sort_values(by=['value'], ascending=False)
table = table[table['metric'] == 'roc_auc_score']
print(table)
print(f"The best model is {table['model'].iloc[0]} with {table['value'].iloc[0]} as {table['metric'].iloc[0]}")

# Predict test data using the best model
test_predictions = eval(table['model'].iloc[0]+"_best_estimator").predict(X_test)
print('Predictions from best model are stored in test_predictions')
